# Out of Band Protocol - Receiver

The out of band protocol allows agents to exchange messages without requiring a DIDComm channel. This can be used to establish a connection, request a presentation or issue a credential. 

The RFC is described [here](https://github.com/hyperledger/aries-rfcs/tree/master/features/0434-outofband)

Begin in the [sender notebook](http://localhost:8889/lab/tree/4%20Advanced%20Concepts/Part%203%20-%20Out%20of%20Band%20Protocol.ipynb).

## 5. Initialise the Controller

In [ ]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(admin_url=ADMIN_URL)

In [ ]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 6. Configure Connection Listeners

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connections_handler(payload):
    print("Connection Webhook : ", payload)
    
connections_listener = {
    "topic": "connections",
    "handler": connections_handler
}


agent_controller.register_listeners([connections_listener], defaults=True)

## 7. Receive Out of Band Invitation

**You should have copied this from the Sender notebook (Step 3)**

In [ ]:
import json
oob_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/out-of-band/1.0/invitation', '@id': 'e536ed97-2e32-4133-bf20-7ce998739267', 'label': 'Alice', 'handshake_protocols': ['https://didcomm.org/connections/1.0/invitation', 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation'], 'request~attach': [], 'service': [{'id': '#inline', 'type': 'did-communication', 'recipientKeys': ['did:key:z6Mkn6wscbEJxHZ4Fnf31DwuSfADcWqh6Ncc3fpNJTYHYZHP'], 'routingKeys': [], 'serviceEndpoint': 'http://172.17.0.1:8020'}]}
response = await agent_controller.oob.receive_invitation(oob_invite)
print(response)

## 8. Get Connection

In [ ]:
connection_id = response["connection_id"]
print(connection_id)

connection = await agent_controller.connections.get_connection(connection_id)

print(connection)
print("Connection State : ", connection["state"])

## 9. Send Trust Ping

This moves the connection to the active state

In [ ]:
await agent_controller.messaging.trust_ping(connection_id, "active")

## End Tutorial


In [ ]:
await agent_controller.terminate()